<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-31 15:27:07
-------------------
qualified stocks: 95
with latest results: 30
still star stocks: 20
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  4.51 L
Current:  1.37 C
-------------------
Today PnL: -69.66 K (-0.51%)
Current PnL: -17.58 L (-11.96%)
CY Booked + Current PnL: -9.55 L (-6.49%)
-------------------
Total profit:  3.22 L
Total loss:  -20.80 L
-------------------
Total Booked + Current PnL: 17.43 L (14.52%)
Total Booked PnL: 35.01 L (29.16%)
Curr Year Booked PnL: 8.03 L (5.86%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 75.80 L (55.31%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-17.40,H-LC,6.60,112500.0,8934.0,11779.0,0.67,8.63,10.47,20.00,15.0,0.76,0.85,23.91,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,15.77,H-LC,9.67,103806.0,10373.0,12218.0,0.35,11.10,11.77,24.18,31.0,0.85,0.78,26.05,XY25,ATH,FMCG
39,HINDALCO,651.95,761.55,-20.73,H-LC,5.26,109304.0,4992.0,12548.0,-0.82,4.79,11.48,16.81,10.0,0.40,0.82,21.60,X5K,ATH,METALS
74,SIS,477.00,477.00,2335.75,H-SC,6.59,65535.0,-15555.0,15558.0,0.34,-19.18,23.74,0.00,157.0,-1.00,0.49,32.82,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-12.46,M-LC,4.34,212437.0,10958.0,16251.0,0.31,5.44,7.65,13.51,62.0,0.67,1.60,18.72,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.99,M-SC,41.84,64463.0,-543.0,151475.0,-1.16,-0.83,234.98,232.18,199.0,-0.00,0.49,0.15,XY24,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,19.76,100448.0,-515.0,95044.0,3.33,-0.51,94.62,93.63,149.0,-0.01,0.76,49.62,SR,ATH,CHEMICALS
14,BERGEPAINT,561.33,680.00,-19.20,H-MC,10.39,228582.0,1243.0,46814.0,-1.83,0.55,20.48,21.14,107.0,0.03,1.72,27.98,XY24,NTT,PAINTS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-19.10,H-LC,1.74,212816.0,8316.0,33093.0,-0.38,4.07,15.55,20.25,9.0,0.25,1.61,13.15,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.0,-9.80,H-LC,2.49,187075.0,-13431.0,62202.0,-0.81,-6.70,33.25,24.33,30.0,-0.22,1.41,7.88,X40N,BTT,FINANCE
56,LTIM,5564.16,7230.2,-4.50,H-LC,2.69,183816.0,-16494.0,76467.0,-0.65,-8.23,41.60,29.94,24.0,-0.22,1.39,26.99,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-18.07,H-LC,3.23,162860.0,-38526.0,126135.0,-0.86,-19.13,77.45,43.51,19.0,-0.31,1.23,13.37,X40,BTT,PAINTS
40,HINDUNILVR,2413.81,2723.0,-9.74,H-LC,4.24,282576.0,12229.0,22408.0,3.51,4.52,7.93,12.81,28.0,0.55,2.13,18.18,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-319.59,H-SC,20.66,93327.0,-39959.0,124554.0,3.80,-29.98,133.46,63.47,147.0,-0.32,0.70,56.44,XR,NTT,CHEMICALS
40,HINDUNILVR,2413.81,2723.00,-9.74,H-LC,4.24,282576.0,12229.0,22408.0,3.51,4.52,7.93,12.81,28.0,0.55,2.13,18.18,XY25,NTT,FMCG
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,19.76,100448.0,-515.0,95044.0,3.33,-0.51,94.62,93.63,149.0,-0.01,0.76,49.62,SR,ATH,CHEMICALS
51,JIOFIN,310.58,387.00,7.44,H-LC,13.68,230195.0,12789.0,40698.0,2.67,5.88,17.68,24.61,50.0,0.31,1.74,63.67,XY24,BTT,FINANCE
68,SAMMAANCAP,170.35,326.00,-165.91,M-SC,4.26,76020.0,-26190.0,119579.0,2.28,-25.62,157.30,91.37,210.0,-0.22,0.57,23.41,XY25,NTT,FINANCE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,RELAXO,902.64,1176.00,-36.38,H-SC,13.77,57542.0,-57996.0,92982.0,-6.92,-50.20,161.59,30.28,134.0,-0.62,0.43,12.42,X40N,NTT,FOOTWEAR
18,BSOFT,628.85,836.99,-18.64,H-SC,12.55,68268.0,-41781.0,78208.0,-5.94,-37.97,114.56,33.10,129.0,-0.53,0.52,13.44,XR,ATH,IT
16,BLUSPRING,226.45,94.95,NaN,NaN,6.81,19173.0,-30420.0,1620.0,-4.48,-61.34,8.45,-58.07,261.0,-18.78,0.14,28.58,XY24,ATH,MISC
20,CAMS,3643.00,5226.82,-7.03,H-SC,0.29,104434.0,2430.0,41920.0,-4.12,2.38,40.14,43.48,120.0,0.06,0.79,19.56,X40N,ATH,MISC
93,WHIRLPOOL,1219.98,2270.00,-40.32,M-SC,7.51,100342.0,8844.0,69908.0,-3.33,9.67,69.67,86.07,211.0,0.13,0.76,45.16,XR,NTT,DURABLES


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SYMPHONY,1306.42,1306.0,-31.46,M-SC,8.89,146052.0,-25089.0,25033.0,-2.20,-14.66,17.14,-0.03,197.0,-1.00,1.10,5.69,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,6.15,148800.0,-25759.0,25787.0,-0.23,-14.76,17.33,0.02,135.0,-1.00,1.12,29.03,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.88,119799.0,-22680.0,23301.0,1.67,-15.92,19.45,0.44,163.0,-0.97,0.90,41.40,OX40N,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-18.82,H-MC,1.85,85134.0,-17518.0,17512.0,-0.33,-17.07,20.57,-0.00,101.0,-1.00,0.64,29.41,OX40N,NTT,IT
85,TTKPRESTIG,769.29,770.0,-9.79,H-SC,12.04,82471.0,-18306.0,18399.0,0.41,-18.16,22.31,0.09,145.0,-0.99,0.62,8.35,OX40N,NTT,DURABLES


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TCS,3794.03,4998.00,-27.38,H-LC,13.45,236605.0,-59329.0,153249.0,-0.66,-20.05,64.77,31.73,1.0,-0.39,1.79,0.00,X40,BTT,IT
32,ENRIN,1377.95,3229.70,NaN,NaN,2.88,80742.0,46293.0,0.0,0.09,134.38,0.00,134.38,2.0,inf,0.61,23.47,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-7.32,H-LC,11.92,281401.0,16361.0,80284.0,0.18,6.17,28.53,36.46,3.0,0.20,2.12,20.18,X40N,ATH,FMCG
48,INFY,1461.46,2275.00,-15.65,H-LC,7.13,270093.0,8492.0,137126.0,-0.66,3.25,50.77,55.67,4.0,0.06,2.04,9.55,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.77,H-LC,7.50,214832.0,-1687.0,94591.0,-0.82,-0.78,44.03,42.91,5.0,-0.02,1.62,12.29,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.11,50889.0,9634.0,7801.0,-2.06,23.35,15.33,42.26,198.0,1.23,0.38,19.81,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,18.72,H-SC,12.08,152111.0,25911.0,20261.0,0.38,20.53,13.32,36.59,156.0,1.28,1.15,35.62,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3229.70,NaN,NaN,2.88,80742.0,46293.0,0.0,0.09,134.38,0.00,134.38,2.0,inf,0.61,23.47,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,26.30,H-SC,12.79,129226.0,3929.0,55154.0,-0.38,3.14,42.68,47.15,144.0,0.07,0.97,61.67,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,18.72,H-SC,12.08,152111.0,25911.0,20261.0,0.38,20.53,13.32,36.59,156.0,1.28,1.15,35.62,XY24,NTT,HEALTHCARE
26,DABUR,505.20,735.00,-4.85,H-MC,4.92,206134.0,9106.0,80516.0,1.25,4.62,39.06,45.49,100.0,0.11,1.56,18.27,XY24,BTT,FMCG
20,CAMS,3643.00,5226.82,-7.03,H-SC,0.29,104434.0,2430.0,41920.0,-4.12,2.38,40.14,43.48,120.0,0.06,0.79,19.56,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-7.42,H-SC,9.77,173680.0,9010.0,58964.0,-1.02,5.47,33.95,41.28,119.0,0.15,1.31,13.89,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3229.70,NaN,NaN,2.88,80742.0,46293.0,0.0,0.09,134.38,0.00,134.38,2.0,inf,0.61,23.47,AR,ATH,ELECTRICAL
93,WHIRLPOOL,1219.98,2270.00,-40.32,M-SC,7.51,100342.0,8844.0,69908.0,-3.33,9.67,69.67,86.07,211.0,0.13,0.76,45.16,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,13.16,137238.0,13902.0,119864.0,0.16,11.27,87.34,108.45,118.0,0.12,1.04,36.14,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,26.30,H-SC,12.79,129226.0,3929.0,55154.0,-0.38,3.14,42.68,47.15,144.0,0.07,0.97,61.67,MH,ATH,POWER
94,WIPRO,243.46,420.00,-10.12,M-LC,5.55,154070.0,3125.0,106324.0,-0.68,2.07,69.01,72.51,55.0,0.03,1.16,8.22,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,6.81,19173.0,-30420.0,1620.0,-4.48,-61.34,8.45,-58.07,261.0,-18.78,0.14,28.58,XY24,ATH,MISC
74,SIS,477.00,477.00,2335.75,H-SC,6.59,65535.0,-15555.0,15558.0,0.34,-19.18,23.74,0.00,157.0,-1.00,0.49,32.82,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.05,68410.0,-45139.0,85191.0,-1.57,-39.75,124.53,35.27,267.0,-0.53,0.52,67.43,XR,NTT,HOTELS
32,ENRIN,1377.95,3229.70,NaN,NaN,2.88,80742.0,46293.0,0.0,0.09,134.38,0.00,134.38,2.0,inf,0.61,23.47,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-18.70,H-SC,16.82,92400.0,-5580.0,27452.0,0.00,-5.70,29.71,22.32,151.0,-0.20,0.70,34.01,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.11,50889.0,9634.0,7801.0,-2.06,23.35,15.33,42.26,198.0,1.23,0.38,19.81,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,18.72,H-SC,12.08,152111.0,25911.0,20261.0,0.38,20.53,13.32,36.59,156.0,1.28,1.15,35.62,XY24,NTT,HEALTHCARE
16,BLUSPRING,226.45,94.95,NaN,NaN,6.81,19173.0,-30420.0,1620.0,-4.48,-61.34,8.45,-58.07,261.0,-18.78,0.14,28.58,XY24,ATH,MISC
91,VBL,492.64,672.28,-7.32,H-LC,11.92,281401.0,16361.0,80284.0,0.18,6.17,28.53,36.46,3.0,0.20,2.12,20.18,X40N,ATH,FMCG
72,SHALBY,261.39,327.00,965.39,M-SC,9.07,144934.0,-36209.0,81670.0,-0.37,-19.99,56.35,25.10,233.0,-0.44,1.09,16.15,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.77
1,25,41.73
2,50,70.82


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.11
LC    35.61
MC    22.13
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.77
X40N     14.95
X40      12.64
XY25     10.93
XR        9.04
AR        8.29
OX40N     7.84
X5K       2.21
MH        1.71
X200      1.39
SR        1.21
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.46
H-SC    27.43
H-MC    19.20
M-SC    12.23
M-LC     7.10
M-MC     2.59
L-SC     1.45
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.65,-0.74,34.44
IT,11.78,-16.49,73.88
FINANCE,8.40,-17.10,59.58
BANKS,7.47,-14.92,61.81
PAINTS,5.73,-14.75,37.36
ELECTRICAL,5.44,-2.16,50.75
MISC,5.42,-20.67,83.22
HEALTHCARE,4.71,3.19,22.52
AUTO,4.33,-17.68,68.88


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2770882.0,25
XR,1094573.0,13
AR,1047857.0,10
X40,778083.0,10
X40N,569606.0,12
XY25,488869.0,8
OX40N,336572.0,10
SR,183485.0,2
X5K,145814.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2686221.0,30
M-SC,1349863.0,15
H-LC,1225046.0,20
H-MC,1160761.0,15
M-LC,437019.0,5
M-MC,334256.0,2
L-SC,270170.0,3
L-MC,57076.0,1
L-LC,50467.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      925031.0      7
M-SC       XY24      827774.0      7
H-SC       AR        612546.0      4
           XR        513014.0      6
H-LC       X40       509775.0      6
H-MC       XY24      452524.0      4
M-MC       XY24      334256.0      2
H-LC       X40N      273400.0      6
H-SC       X40N      220632.0      4
H-LC       AR        218114.0      2
H-MC       XY25      208528.0      2
           X40       193135.0      3
H-SC       SR        183485.0      2
L-SC       XR        182721.0      2
M-LC       XY24      181178.0      2
H-SC       OX40N     176359.0      6
M-SC       AR        157918.0      2
M-LC       X5K       133266.0      1
H-MC       XR        121271.0      1
M-SC       XY25      119579.0      1
           XR        114167.0      2
M-LC       XR        106324.0      1
H-LC       XY25       94044.0      3
L-SC       OX40N      87449.0      1
H-LC       X200       76467.0      1
H-MC       X40N       75574.0      2
M-SC       X40        75173.0      1
H-MC       AR         59279.0      1
L-MC       XR         57076.0      1
M-SC       OX40N      55252.0      2
H-SC       MH         55154.0      1
L-LC       XY25       50467.0      1
H-LC       XY24       40698.0      1
H-MC       MH         32938.0      1
           OX40N      17512.0      1
M-LC       XY25       16251.0      1
H-LC       X5K        12548.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
